In [1]:
import nltk
from nltk.book import *
from nltk.corpus import stopwords

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


## Recursos léxicos (lexicons)

- Son collecciones de palabras o frases que tienen asociadas etiquetas o meta-información de algún tipo (POS tags, significados gramaticales, etc...)

- POS (Part Of Speech), también llamado etiquetado gramatical o etiquetado de palabras por categorias, consite en etiquetar la categoria gramatical a la que pertenece cada palabra en un volumen de texto, siendo las categorías: 

    1. Sustantivos
    2. Adjetivos
    3. Artículos
    4. Pronombres
    5. Verbos
    6. Adverbios
    7. Interjecciones
    8. Preposiciones
    9. Conjunciones

In [7]:
# Vocabularios: palabras únicas en un corpus
vocab = sorted(set(text1))
vocab[1000:1010]

['Crew',
 'Crish',
 'Crockett',
 'Cross',
 'Crossed',
 'Crossing',
 'Crotch',
 'Crowding',
 'Crown',
 'Crozetts']

Un vocabulario normal como este es un recurso léxico no enriquezido (no tiene descripción).

In [13]:
# Distribuciones:
word_freq = FreqDist(text1)
word_freq.most_common(5)

[(',', 18713), ('the', 13721), ('.', 6862), ('of', 6536), ('and', 6024)]

Un FreqDist es un lexicon, ya que tiene un texto con información de cuanto se repite dicho texto. 

In [15]:
# Stopwords. Palabras muy usadas que son filtradas de NLP (useless words)
stopwords.words('spanish')[:15]

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con']

## Fracción de Stopwords en un corpus

In [22]:
def stopwords_percentage(text):
    stopwd = stopwords.words('english')
    content = [word for word in text if word.lower() not in stopwd]
    
    return 1 - len(content)/len(text)

In [23]:
stopwords_percentage(text1)

0.4137045230600531

Esto es, el 41% de todo el libro de Movy-Dick son palabras sin mucha importancia

## Lexicons enriquecidos (listas comparativas de palabras)
- Construyendo diccionarios para traducción de palabras en diferentes idiomas

In [24]:
from nltk.corpus import swadesh
print(swadesh.fileids())

['be', 'bg', 'bs', 'ca', 'cs', 'cu', 'de', 'en', 'es', 'fr', 'hr', 'it', 'la', 'mk', 'nl', 'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'sr', 'sw', 'uk']


In [27]:
print(swadesh.words('en')[:10])

['I', 'you (singular), thou', 'he', 'we', 'you (plural)', 'they', 'this', 'that', 'here', 'there']


In [32]:
esp2france = swadesh.entries(['es','fr'])
print(france2esp[:10])

[('je', 'yo'), ('tu, vous', 'tú, usted'), ('il', 'él'), ('nous', 'nosotros'), ('vous', 'vosotros, ustedes'), ('ils, elles', 'ellos, ellas'), ('ceci', 'este'), ('cela', 'ese, aquel'), ('ici', 'aquí, acá'), ('là', 'ahí, allí, allá')]


In [34]:
translate = dict(esp2france)
translate['perro']

'chien'

In [35]:
translate['tirar']

'jeter'